In [ ]:
from pathlib import Path

from tqdm import tqdm

import deepecohab

#### Make project

Uses example data provided in the repository. Currently only default ecohab layout, more examples soon...

In [ ]:
example_dir = Path.cwd()

In [ ]:
config_path = deepecohab.create_ecohab_project(
	project_location=example_dir,  # Location where the project will be created
	experiment_name="test",  # Name of the project
	data_path=example_dir / "test_data",  # Folder with ecohab txt files
	light_phase_start="00:00:00",
	dark_phase_start="12:00:00",
	animal_ids=[
		"75DFFF1904",
		"352DE61A04",
		"9A44E61A04",
		"0627A81904",
		"A4939C1A04",
		"7376E61A04",
		"956BA81904",
		"325CE61A04",
		"85EDFF1904",
	],
)

#### Make structure

In [ ]:
lf = deepecohab.get_ecohab_data_structure(
	config_path,
	fname_prefix="20",
	sanitize_animal_ids=True,
	min_antenna_crossings=100,
	timezone="Europe/Warsaw",
)

#### Run antenna analysis pipeline

In [ ]:
pipeline = deepecohab.df_registry.run_pipeline(
	config_path,
	minimum_time=2,
	chasing_time_window=[0.1, 1.2],
)

with tqdm(total=len(deepecohab.df_registry.analysis_steps)) as pbar:
	for step_name, _, _ in pipeline:
		pbar.set_description(f"Processing {step_name}")
		pbar.update(1)

print("Done!")

### Load data you're interested in

In [ ]:
deepecohab.df_registry.list_available()

In [ ]:
chasings_df = deepecohab.load_ecohab_data(config_path, 'chasings_df', return_df=True)

### Generate plots in the notebook

In [ ]:
deepecohab.plot_registry.list_available()

In [ ]:
cfg = deepecohab.read_config(config_path)
plt_cfg = deepecohab.PlotConfig(
    store = {
        "chasings_df": chasings_df,
    },
    days_range=[2,5],
    agg_switch='sum',
    animals=cfg['animal_ids'],
    animal_colors=deepecohab.utils.auxfun_plots.color_sampling(cfg['animal_ids']),
)

deepecohab.plot_registry.get_plot("chasings-line", plt_cfg)